In [3]:
"""
Classification Case Study: Customer Churn Prediction
=====================================================
A comprehensive webinar demonstrating various classification algorithms:
- Logistic Regression
- Decision Tree
- K-Nearest Neighbors (KNN)
- Support Vector Machine (SVM)
- Instance-Based Learning
- Ensemble Methods (Random Forest, Gradient Boosting, Voting Classifier)

Dataset: IBM Telco Customer Churn (Real Dataset)
Download from: https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv

Or use: https://www.kaggle.com/datasets/blastchar/telco-customer-churn
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    VotingClassifier,
    AdaBoostClassifier,
    BaggingClassifier
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve, auc,
    precision_recall_curve, average_precision_score
)
import warnings
warnings.filterwarnings('ignore')

# Set style for visualizations
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# =============================================================================
# SECTION 1: DATA LOADING AND PREPARATION
# =============================================================================

def load_telco_data(filepath=None):
    """
    Load the IBM Telco Customer Churn dataset.

    If filepath is provided, loads from local file.
    Otherwise, attempts to download from GitHub.

    Download the dataset manually from:
    https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv

    Or from Kaggle:
    https://www.kaggle.com/datasets/blastchar/telco-customer-churn
    """
    if filepath:
        print(f"Loading data from: {filepath}")
        df = pd.read_csv(filepath)
    else:
        # Try to download from GitHub
        url = "https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv"
        try:
            print(f"Attempting to download from: {url}")
            df = pd.read_csv(url)
            print("Successfully downloaded dataset!")
        except Exception as e:
            print(f"Could not download dataset: {e}")
            print("\nPlease download the dataset manually from:")
            print("https://www.kaggle.com/datasets/blastchar/telco-customer-churn")
            print("\nOr use the direct GitHub link:")
            print("https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv")
            print("\nThen run: load_telco_data('path/to/Telco-Customer-Churn.csv')")
            return None

    return df

def explore_data(df):
    """
    Perform exploratory data analysis on the dataset.
    """
    print("\n" + "="*60)
    print("EXPLORATORY DATA ANALYSIS")
    print("="*60)

    print(f"\nDataset Shape: {df.shape}")
    print(f"Total Customers: {len(df):,}")

    print("\nColumn Names:")
    print(df.columns.tolist())

    print("\nData Types:")
    print(df.dtypes)

    print("\nMissing Values:")
    print(df.isnull().sum()[df.isnull().sum() > 0])

    print("\nChurn Distribution:")
    print(df['Churn'].value_counts())
    print(f"\nChurn Rate: {df['Churn'].value_counts(normalize=True).get('Yes', df['Churn'].value_counts(normalize=True).get(1, 0)):.2%}")

    return df

def preprocess_telco_data(df):
    """
    Preprocess the IBM Telco Customer Churn dataset.
    """
    print("\n" + "="*60)
    print("DATA PREPROCESSING")
    print("="*60)

    df_processed = df.copy()

    # Drop customerID (not useful for prediction)
    if 'customerID' in df_processed.columns:
        df_processed = df_processed.drop('customerID', axis=1)
        print("Dropped customerID column")

    # Handle TotalCharges - convert to numeric and handle missing/empty values
    if 'TotalCharges' in df_processed.columns:
        df_processed['TotalCharges'] = pd.to_numeric(df_processed['TotalCharges'], errors='coerce')
        # Fill missing TotalCharges with tenure * MonthlyCharges
        mask = df_processed['TotalCharges'].isnull()
        if mask.any():
            df_processed.loc[mask, 'TotalCharges'] = df_processed.loc[mask, 'tenure'] * df_processed.loc[mask, 'MonthlyCharges']
            print(f"Filled {mask.sum()} missing TotalCharges values")

    # Convert Churn to binary
    if df_processed['Churn'].dtype == 'object':
        df_processed['Churn'] = (df_processed['Churn'] == 'Yes').astype(int)
        print("Converted Churn to binary (Yes=1, No=0)")

    # Encode categorical variables
    label_encoders = {}
    categorical_cols = df_processed.select_dtypes(include=['object']).columns

    print(f"\nEncoding {len(categorical_cols)} categorical columns:")
    for col in categorical_cols:
        le = LabelEncoder()
        df_processed[col] = le.fit_transform(df_processed[col].astype(str))
        label_encoders[col] = le
        print(f"  - {col}: {len(le.classes_)} unique values")

    # Check for any remaining missing values
    if df_processed.isnull().sum().sum() > 0:
        print("\nDropping rows with remaining missing values...")
        df_processed = df_processed.dropna()

    print(f"\nFinal dataset shape: {df_processed.shape}")

    return df_processed, label_encoders

def prepare_data_splits(df_processed, test_size=0.2, random_state=42):
    """
    Split data into training and testing sets.
    """
    X = df_processed.drop('Churn', axis=1)
    y = df_processed['Churn']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    print(f"\nTraining set: {len(X_train):,} samples")
    print(f"Test set: {len(X_test):,} samples")
    print(f"Features: {X_train.shape[1]}")

    return X_train, X_test, y_train, y_test, X_train_scaled, X_test_scaled, scaler

# =============================================================================
# SECTION 2: MODEL IMPLEMENTATIONS
# =============================================================================

def train_logistic_regression(X_train, y_train, X_test, y_test):
    """
    Train and evaluate Logistic Regression model.
    """
    print("\n" + "="*60)
    print("LOGISTIC REGRESSION")
    print("="*60)

    # Model with regularization tuning
    lr = LogisticRegression(random_state=42, max_iter=1000, C=0.5)
    lr.fit(X_train, y_train)

    # Predictions
    y_pred = lr.predict(X_test)
    y_prob = lr.predict_proba(X_test)[:, 1]

    # Evaluation
    print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")

    # Feature importance (coefficients)
    if hasattr(X_train, 'columns'):
        print("\nTop 5 Feature Coefficients (by absolute value):")
        feature_importance = pd.DataFrame({
            'feature': X_train.columns,
            'coefficient': lr.coef_[0]
        }).sort_values('coefficient', key=abs, ascending=False)
        print(feature_importance.head())

    return lr, y_pred, y_prob

def train_decision_tree(X_train, y_train, X_test, y_test):
    """
    Train and evaluate Decision Tree model with hyperparameter tuning.
    """
    print("\n" + "="*60)
    print("DECISION TREE")
    print("="*60)

    # Hyperparameter tuning
    param_grid = {
        'max_depth': [3, 5, 7, 10],
        'min_samples_split': [2, 5, 10, 20],
        'min_samples_leaf': [1, 2, 4, 8]
    }

    dt = DecisionTreeClassifier(random_state=42)
    grid_search = GridSearchCV(dt, param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_dt = grid_search.best_estimator_
    print(f"\nBest Parameters: {grid_search.best_params_}")

    # Predictions
    y_pred = best_dt.predict(X_test)
    y_prob = best_dt.predict_proba(X_test)[:, 1]

    # Evaluation
    print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")

    # Feature importance
    if hasattr(X_train, 'columns'):
        print("\nTop 5 Feature Importance:")
        feature_importance = pd.DataFrame({
            'feature': X_train.columns,
            'importance': best_dt.feature_importances_
        }).sort_values('importance', ascending=False)
        print(feature_importance.head())

    return best_dt, y_pred, y_prob

def train_knn(X_train_scaled, y_train, X_test_scaled, y_test):
    """
    Train and evaluate K-Nearest Neighbors model.
    """
    print("\n" + "="*60)
    print("K-NEAREST NEIGHBORS (KNN)")
    print("="*60)

    # Find optimal K (odd numbers from 3 to 29 to avoid ties)
    k_range = range(3, 30, 2)
    cv_scores = []

    print("\nSearching for optimal K...")
    for k in k_range:
        knn = KNeighborsClassifier(n_neighbors=k, weights='distance')
        scores = cross_val_score(knn, X_train_scaled, y_train, cv=5, scoring='f1')
        cv_scores.append(scores.mean())

    best_k = list(k_range)[np.argmax(cv_scores)]
    print(f"Optimal K: {best_k}")

    # Train with best K
    knn = KNeighborsClassifier(n_neighbors=best_k, weights='distance')
    knn.fit(X_train_scaled, y_train)

    # Predictions
    y_pred = knn.predict(X_test_scaled)
    y_prob = knn.predict_proba(X_test_scaled)[:, 1]

    # Evaluation
    print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")

    return knn, y_pred, y_prob, k_range, cv_scores

def train_svm(X_train_scaled, y_train, X_test_scaled, y_test):
    """
    Train and evaluate Support Vector Machine model.
    """
    print("\n" + "="*60)
    print("SUPPORT VECTOR MACHINE (SVM)")
    print("="*60)

    # Train with different kernels
    kernels = ['linear', 'rbf', 'poly']
    results = {}

    for kernel in kernels:
        print(f"\nTraining {kernel} kernel...")
        svm = SVC(kernel=kernel, probability=True, random_state=42, C=1.0)
        svm.fit(X_train_scaled, y_train)
        y_pred = svm.predict(X_test_scaled)
        results[kernel] = {
            'model': svm,
            'accuracy': accuracy_score(y_test, y_pred),
            'f1': f1_score(y_test, y_pred)
        }
        print(f"  {kernel.upper()} - Accuracy: {results[kernel]['accuracy']:.4f}, F1: {results[kernel]['f1']:.4f}")

    # Select best kernel
    best_kernel = max(results.keys(), key=lambda k: results[k]['f1'])
    best_svm = results[best_kernel]['model']

    print(f"\nBest Kernel: {best_kernel}")

    # Final predictions with best model
    y_pred = best_svm.predict(X_test_scaled)
    y_prob = best_svm.predict_proba(X_test_scaled)[:, 1]

    print(f"\nFinal Results ({best_kernel}):")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")

    return best_svm, y_pred, y_prob, results

def train_ensemble_models(X_train, y_train, X_test, y_test, X_train_scaled, X_test_scaled):
    """
    Train and evaluate Ensemble Learning models.
    """
    print("\n" + "="*60)
    print("ENSEMBLE LEARNING")
    print("="*60)

    ensemble_results = {}

    # 1. Random Forest
    print("\n--- Random Forest ---")
    rf = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        min_samples_split=5,
        random_state=42,
        n_jobs=-1
    )
    rf.fit(X_train, y_train)
    y_pred_rf = rf.predict(X_test)
    y_prob_rf = rf.predict_proba(X_test)[:, 1]
    ensemble_results['Random Forest'] = {
        'model': rf,
        'y_pred': y_pred_rf,
        'y_prob': y_prob_rf,
        'accuracy': accuracy_score(y_test, y_pred_rf),
        'f1': f1_score(y_test, y_pred_rf)
    }
    print(f"Accuracy: {ensemble_results['Random Forest']['accuracy']:.4f}")
    print(f"F1 Score: {ensemble_results['Random Forest']['f1']:.4f}")

    # 2. Gradient Boosting
    print("\n--- Gradient Boosting ---")
    gb = GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        random_state=42
    )
    gb.fit(X_train, y_train)
    y_pred_gb = gb.predict(X_test)
    y_prob_gb = gb.predict_proba(X_test)[:, 1]
    ensemble_results['Gradient Boosting'] = {
        'model': gb,
        'y_pred': y_pred_gb,
        'y_prob': y_prob_gb,
        'accuracy': accuracy_score(y_test, y_pred_gb),
        'f1': f1_score(y_test, y_pred_gb)
    }
    print(f"Accuracy: {ensemble_results['Gradient Boosting']['accuracy']:.4f}")
    print(f"F1 Score: {ensemble_results['Gradient Boosting']['f1']:.4f}")

    # 3. AdaBoost
    print("\n--- AdaBoost ---")
    ada = AdaBoostClassifier(n_estimators=100, learning_rate=0.5, random_state=42)
    ada.fit(X_train, y_train)
    y_pred_ada = ada.predict(X_test)
    y_prob_ada = ada.predict_proba(X_test)[:, 1]
    ensemble_results['AdaBoost'] = {
        'model': ada,
        'y_pred': y_pred_ada,
        'y_prob': y_prob_ada,
        'accuracy': accuracy_score(y_test, y_pred_ada),
        'f1': f1_score(y_test, y_pred_ada)
    }
    print(f"Accuracy: {ensemble_results['AdaBoost']['accuracy']:.4f}")
    print(f"F1 Score: {ensemble_results['AdaBoost']['f1']:.4f}")

    # 4. Bagging Classifier
    print("\n--- Bagging Classifier ---")
    bagging = BaggingClassifier(n_estimators=50, random_state=42, n_jobs=-1)
    bagging.fit(X_train, y_train)
    y_pred_bag = bagging.predict(X_test)
    y_prob_bag = bagging.predict_proba(X_test)[:, 1]
    ensemble_results['Bagging'] = {
        'model': bagging,
        'y_pred': y_pred_bag,
        'y_prob': y_prob_bag,
        'accuracy': accuracy_score(y_test, y_pred_bag),
        'f1': f1_score(y_test, y_pred_bag)
    }
    print(f"Accuracy: {ensemble_results['Bagging']['accuracy']:.4f}")
    print(f"F1 Score: {ensemble_results['Bagging']['f1']:.4f}")

    # 5. Voting Classifier
    print("\n--- Voting Classifier (Soft Voting) ---")
    voting_clf = VotingClassifier(
        estimators=[
            ('lr', LogisticRegression(random_state=42, max_iter=1000)),
            ('rf', RandomForestClassifier(n_estimators=50, max_depth=8, random_state=42)),
            ('gb', GradientBoostingClassifier(n_estimators=50, max_depth=4, random_state=42))
        ],
        voting='soft'
    )
    voting_clf.fit(X_train, y_train)
    y_pred_vote = voting_clf.predict(X_test)
    y_prob_vote = voting_clf.predict_proba(X_test)[:, 1]
    ensemble_results['Voting Classifier'] = {
        'model': voting_clf,
        'y_pred': y_pred_vote,
        'y_prob': y_prob_vote,
        'accuracy': accuracy_score(y_test, y_pred_vote),
        'f1': f1_score(y_test, y_pred_vote)
    }
    print(f"Accuracy: {ensemble_results['Voting Classifier']['accuracy']:.4f}")
    print(f"F1 Score: {ensemble_results['Voting Classifier']['f1']:.4f}")

    return ensemble_results

# =============================================================================
# SECTION 3: EVALUATION & VISUALIZATION
# =============================================================================

def plot_confusion_matrices(y_test, predictions_dict, save_path='confusion_matrices.png'):
    """Plot confusion matrices for all models."""
    n_models = len(predictions_dict)
    n_cols = 3
    n_rows = (n_models + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, 4*n_rows))
    axes = axes.flatten()

    for idx, (name, y_pred) in enumerate(predictions_dict.items()):
        if idx < len(axes):
            cm = confusion_matrix(y_test, y_pred)
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                       xticklabels=['No Churn', 'Churn'],
                       yticklabels=['No Churn', 'Churn'])
            axes[idx].set_title(f'{name}', fontsize=11, fontweight='bold')
            axes[idx].set_xlabel('Predicted')
            axes[idx].set_ylabel('Actual')

    for idx in range(n_models, len(axes)):
        axes[idx].set_visible(False)

    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"Saved: {save_path}")

def plot_roc_curves(y_test, probabilities_dict, save_path='roc_curves.png'):
    """Plot ROC curves for all models."""
    plt.figure(figsize=(10, 8))
    colors = plt.cm.tab10(np.linspace(0, 1, len(probabilities_dict)))

    for (name, y_prob), color in zip(probabilities_dict.items(), colors):
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, color=color, lw=2, label=f'{name} (AUC = {roc_auc:.3f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('ROC Curves Comparison', fontsize=14, fontweight='bold')
    plt.legend(loc='lower right', fontsize=9)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"Saved: {save_path}")

def plot_precision_recall_curves(y_test, probabilities_dict, save_path='pr_curves.png'):
    """Plot Precision-Recall curves for all models."""
    plt.figure(figsize=(10, 8))
    colors = plt.cm.tab10(np.linspace(0, 1, len(probabilities_dict)))

    for (name, y_prob), color in zip(probabilities_dict.items(), colors):
        precision, recall, _ = precision_recall_curve(y_test, y_prob)
        avg_precision = average_precision_score(y_test, y_prob)
        plt.plot(recall, precision, color=color, lw=2,
                label=f'{name} (AP = {avg_precision:.3f})')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall', fontsize=12)
    plt.ylabel('Precision', fontsize=12)
    plt.title('Precision-Recall Curves Comparison', fontsize=14, fontweight='bold')
    plt.legend(loc='lower left', fontsize=9)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"Saved: {save_path}")

def plot_model_comparison(metrics_dict, save_path='model_comparison.png'):
    """Create a bar chart comparing all models across metrics."""
    metrics_df = pd.DataFrame(metrics_dict).T

    fig, axes = plt.subplots(1, 4, figsize=(16, 5))
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
    colors = ['#3498db', '#2ecc71', '#e74c3c', '#9b59b6']

    for idx, (metric, color) in enumerate(zip(metrics, colors)):
        values = metrics_df[metric].sort_values(ascending=True)
        axes[idx].barh(values.index, values.values, color=color, alpha=0.8)
        axes[idx].set_xlabel(metric, fontsize=11)
        axes[idx].set_xlim([0.4, 1.0])
        axes[idx].set_title(metric, fontsize=12, fontweight='bold')
        for i, v in enumerate(values.values):
            axes[idx].text(v + 0.01, i, f'{v:.3f}', va='center', fontsize=9)

    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"Saved: {save_path}")

def plot_knn_optimization(k_range, cv_scores, save_path='knn_optimization.png'):
    """Plot KNN optimization curve showing optimal K value."""
    plt.figure(figsize=(10, 6))
    k_list = list(k_range)
    plt.plot(k_list, cv_scores, 'b-o', linewidth=2, markersize=6)
    plt.fill_between(k_list, cv_scores, alpha=0.2)

    best_k = k_list[np.argmax(cv_scores)]
    best_score = max(cv_scores)
    plt.axvline(x=best_k, color='r', linestyle='--', label=f'Optimal K = {best_k}')
    plt.scatter([best_k], [best_score], color='red', s=150, zorder=5, marker='*')

    plt.xlabel('Number of Neighbors (K)', fontsize=12)
    plt.ylabel('Cross-Validation F1 Score', fontsize=12)
    plt.title('KNN Hyperparameter Optimization', fontsize=14, fontweight='bold')
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"Saved: {save_path}")

def plot_feature_importance(model, feature_names, title, save_path):
    """Plot feature importance for tree-based models."""
    if hasattr(model, 'feature_importances_'):
        importance = model.feature_importances_
    elif hasattr(model, 'coef_'):
        importance = np.abs(model.coef_[0])
    else:
        return

    feature_imp = pd.DataFrame({
        'feature': feature_names,
        'importance': importance
    }).sort_values('importance', ascending=True).tail(10)

    plt.figure(figsize=(10, 6))
    plt.barh(feature_imp['feature'], feature_imp['importance'], color='#3498db', alpha=0.8)
    plt.xlabel('Importance', fontsize=12)
    plt.title(f'{title} - Top 10 Features', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"Saved: {save_path}")

def create_summary_table(metrics_dict):
    """Create a summary table of all model metrics."""
    df = pd.DataFrame(metrics_dict).T
    df = df.round(4)
    df.index.name = 'Model'
    return df

# =============================================================================
# SECTION 4: MAIN EXECUTION
# =============================================================================

def main(data_path=None):
    """
    Main function to run the complete classification case study.

    Args:
        data_path: Path to the Telco-Customer-Churn.csv file.
                   If None, will attempt to download from GitHub.
    """
    print("="*70)
    print("CLASSIFICATION CASE STUDY: CUSTOMER CHURN PREDICTION")
    print("Dataset: IBM Telco Customer Churn (Real Data)")
    print("="*70)

    # 1. Load data
    print("\n[1] LOADING DATA...")
    df = load_telco_data(data_path)

    if df is None:
        print("\nERROR: Could not load dataset. Please provide a valid path.")
        return None

    # 2. Explore data
    df = explore_data(df)

    # 3. Preprocess data
    df_processed, label_encoders = preprocess_telco_data(df)
    X_train, X_test, y_train, y_test, X_train_scaled, X_test_scaled, scaler = prepare_data_splits(df_processed)

    feature_names = X_train.columns.tolist()

    # Store all predictions and probabilities
    all_predictions = {}
    all_probabilities = {}
    all_metrics = {}

    # 4. Train models
    print("\n[2] TRAINING MODELS...")

    # Logistic Regression
    lr_model, lr_pred, lr_prob = train_logistic_regression(X_train_scaled, y_train, X_test_scaled, y_test)
    all_predictions['Logistic Regression'] = lr_pred
    all_probabilities['Logistic Regression'] = lr_prob
    all_metrics['Logistic Regression'] = {
        'Accuracy': accuracy_score(y_test, lr_pred),
        'Precision': precision_score(y_test, lr_pred),
        'Recall': recall_score(y_test, lr_pred),
        'F1 Score': f1_score(y_test, lr_pred)
    }

    # Decision Tree
    dt_model, dt_pred, dt_prob = train_decision_tree(X_train, y_train, X_test, y_test)
    all_predictions['Decision Tree'] = dt_pred
    all_probabilities['Decision Tree'] = dt_prob
    all_metrics['Decision Tree'] = {
        'Accuracy': accuracy_score(y_test, dt_pred),
        'Precision': precision_score(y_test, dt_pred),
        'Recall': recall_score(y_test, dt_pred),
        'F1 Score': f1_score(y_test, dt_pred)
    }

    # KNN
    knn_model, knn_pred, knn_prob, k_range, cv_scores = train_knn(X_train_scaled, y_train, X_test_scaled, y_test)
    all_predictions['KNN'] = knn_pred
    all_probabilities['KNN'] = knn_prob
    all_metrics['KNN'] = {
        'Accuracy': accuracy_score(y_test, knn_pred),
        'Precision': precision_score(y_test, knn_pred),
        'Recall': recall_score(y_test, knn_pred),
        'F1 Score': f1_score(y_test, knn_pred)
    }

    # SVM
    svm_model, svm_pred, svm_prob, svm_results = train_svm(X_train_scaled, y_train, X_test_scaled, y_test)
    all_predictions['SVM'] = svm_pred
    all_probabilities['SVM'] = svm_prob
    all_metrics['SVM'] = {
        'Accuracy': accuracy_score(y_test, svm_pred),
        'Precision': precision_score(y_test, svm_pred),
        'Recall': recall_score(y_test, svm_pred),
        'F1 Score': f1_score(y_test, svm_pred)
    }

    # Ensemble Methods
    ensemble_results = train_ensemble_models(X_train, y_train, X_test, y_test, X_train_scaled, X_test_scaled)

    for name, results in ensemble_results.items():
        all_predictions[name] = results['y_pred']
        all_probabilities[name] = results['y_prob']
        all_metrics[name] = {
            'Accuracy': accuracy_score(y_test, results['y_pred']),
            'Precision': precision_score(y_test, results['y_pred']),
            'Recall': recall_score(y_test, results['y_pred']),
            'F1 Score': f1_score(y_test, results['y_pred'])
        }

    # 5. Generate visualizations
    print("\n[3] GENERATING VISUALIZATIONS...")

    plot_confusion_matrices(y_test, all_predictions)
    plot_roc_curves(y_test, all_probabilities)
    plot_precision_recall_curves(y_test, all_probabilities)
    plot_model_comparison(all_metrics)
    plot_knn_optimization(k_range, cv_scores)
    plot_feature_importance(ensemble_results['Random Forest']['model'], feature_names,
                           'Random Forest', 'rf_feature_importance.png')
    plot_feature_importance(ensemble_results['Gradient Boosting']['model'], feature_names,
                           'Gradient Boosting', 'gb_feature_importance.png')

    # 6. Create summary
    print("\n[4] FINAL SUMMARY")
    print("="*70)

    summary_df = create_summary_table(all_metrics)
    print("\nModel Performance Summary:")
    print(summary_df.to_string())

    # Best model
    best_model = summary_df['F1 Score'].idxmax()
    print(f"\n🏆 Best Performing Model (by F1 Score): {best_model}")
    print(f"   F1 Score: {summary_df.loc[best_model, 'F1 Score']:.4f}")
    print(f"   Accuracy: {summary_df.loc[best_model, 'Accuracy']:.4f}")

    # Save summary to CSV
    summary_df.to_csv('model_comparison_summary.csv')
    print("\nSummary saved to: model_comparison_summary.csv")

    return {
        'models': {
            'logistic_regression': lr_model,
            'decision_tree': dt_model,
            'knn': knn_model,
            'svm': svm_model,
            'ensemble': ensemble_results
        },
        'predictions': all_predictions,
        'probabilities': all_probabilities,
        'metrics': all_metrics,
        'summary': summary_df
    }

if __name__ == "__main__":
    # Option 1: Download automatically (requires internet access)
    # results = main()

    # Option 2: Use local file (recommended)
    # Download the dataset from:
    # https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv
    # Then run:
    # results = main('Telco-Customer-Churn.csv')

    # Try automatic download first, fall back to instructions
    results = main()

CLASSIFICATION CASE STUDY: CUSTOMER CHURN PREDICTION
Dataset: IBM Telco Customer Churn (Real Data)

[1] LOADING DATA...
Attempting to download from: https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv
Successfully downloaded dataset!

EXPLORATORY DATA ANALYSIS

Dataset Shape: (7043, 21)
Total Customers: 7,043

Column Names:
['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']

Data Types:
customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
Onli